In [31]:
from ortools.linear_solver import pywraplp
from random import randrange
horario = pywraplp.Solver.CreateSolver('SCIP')

S,T,P,C = 2,10,2,5
disp = []
for c in range(C):
    disp.append([])
    for t in range(T):
        disp[c].append(1)
print(disp)
#A disponibilidade de cada participante, incluindo o lider. Essa disponibilidade   é um conjunto de “slots” 
#representada numa matriz booleana de acessibilidade com uma linha por cada participante 1..C e 
#uma coluna por “slot” 1..T
#disp [C][T] = []

#O conjunto de colaboradores de cada projeto, o seu líder e o número mínimo  de reuniões semanais.
#Nesse caso para um projeto de 0 - P , está associado os colaboradores 
proj = [((1,2,0),1,3),((4,3),4,3)] 

x = {}
for s in range(S):
    x[s]= {}
    for t in range(T):
        x[s][t] = {}
        for p in range(P):
            x[s][t][p] = {}
            for c in range(C):
                x[s][t][p][c] = horario.BoolVar(f'x[{s},{t},{p},{c}]')

def X(s,t,p,c):              # abreviatura
    return x[s][t][p][c] 

#Adiciona condições para que se um colaborador não esteja disponivel para certo slot
#ele não posso ser alocada a esse spot
for c in range(C):
    for t in range(T):
        if disp[c][t] == 0:
            horario.Add((X(s,t,p,c) for p in range(P) for s in range(S))== 0 )

            
#Adiciona condições para que se o lider não esteja disponivel não aconteça o projeto   
for p in range(P):
    l = proj[p][1]
    for t in disp[l]:
        horario.Add((X(s,t,p,c) for s in range(S) for c in proj[p][0])== 0 )

#Adiciona condições para que pelo menos 50% dos colaboradores de um dado projeto esteja presente em um slot 
#alocado a este projeto
for p in range(P):
    for t in range(T):
        horario.Add(sum(X(s,t,p,c) for s in range (S) for c in proj[p][0])  >= len(proj[p][0])/2)
#Adiciona condições para que o número de reunioes por projeto seja maior que o minimo definido
#TALVEZ TENHA Q MUDAR DEPENDENDO DO Q O PROFESSOR RESPONDER SE CERTO COLABORADOR
#PODE PARTICIPAR DE MAIS DO Q UM PROJETO
for p in range(P):
    horario.Add (sum(X(s,t,p,proj[p][1]) for s in range (S) for t in range(T)) >= proj[p][2])

# Cada sala não pode ter mais que uma reunião
for s in range(S):
    for t in range(T):
        horario.Add(sum(X(s,t,p,c) for p in range(P) for c in range(C)) <= 1)
                 
status = horario.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print(":)")

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
